p and q are two large primes.
<br><br>
$n = p * q$
<br><br>
$|\mathbb{Z}_{n}^{*}| = \phi(n) = n * \prod_{\text{p: p is prime and p|n}} (1 - \frac{1}{p})$ (Euler’s phi function) $= n * (1 - \frac{1}{p}) * (1 - \frac{1}{q})$
<br><br>
$ = n * (1 - \frac{1}{q} - \frac{1}{p} + \frac{1}{p*q}) = p * q - p - q + 1$
<br><br>
$ = p * (q - 1) - (q - 1) = (p - 1)*(q - 1)$
<br><br>
$ gcd(e, \phi(n)) = 1 = e * d + \phi(n) * k$ (Theorem 31.2, Introduction to Algorithms 3rd Edition)
<br><br>
$e * d = \phi(n) * (-k) + 1$
<br><br>
$(e * d) \mod \phi(n) = 1$
<br><br>
$d \equiv e^{-1}\mod \phi(n)$ (Multiplicative Inverse of e modulo $\phi(n)$)
<br><br>
$ P = (e, n)$ (RSA public key)
<br><br>
$ S = (d, n)$ (RSA private key)
<br><br>
M is a message.
<br><br>
Sender uses user's public key/address to encrypt the message M: $P(M) = M^{e}(\mod n)$
<br><br>
User uses private key to decrypt the ciphertext: $S(P(M)) = P(M)^{d}(\mod n) \equiv M^{e * d}(\mod n)$
<br><br>
Claim: $S(P(M)) = M^{e * d}(\mod n) \equiv M(\mod n)$
<br><br>
Proof:
<br><br>
$ S(P(M)) = M^{e * d}(\mod n)$
<br><br>
$\because (e * d) \mod \phi(n) = 1$
<br><br>
$\therefore e * d = k * \phi(n) + 1 = k * (p - 1) *(q - 1) + 1$
<br><br>
If $ M \not \equiv 0(\mod p)$,
<br><br>
$ M^{e*d} \equiv M^{k * (p - 1)*(q - 1) + 1}(\mod p)$
<br><br>
$\equiv M * (M^{p - 1})^{k * (q - 1)})(\mod p)$
<br><br>
$\equiv M * (M^{p - 1}(\mod p))^{k * (q - 1)})(\mod p)$
<br><br>
$\equiv M * 1^{k * (q - 1)})(\mod p)$ (Fermat's Theorem)
<br><br>
$\equiv M(\mod p)$
<br><br>
If $ M \equiv 0(\mod p)$, then
<br><br>
$ M^{e*d}\equiv 0(\mod p) \equiv M(\mod p)$
<br><br>
$\therefore M^{e*d}(\mod p) \equiv M(\mod p)$, and similarly,
<br><br>
$M^{e*d}(\mod q) \equiv M(\mod q)$, and
<br><br>
$M^{e*d}(\mod n) \equiv M(\mod n)$ (Chinese remainder theorem)

In [350]:
import random
import binascii

MILLER_RABIN_TRIAL_NUM = 100

def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

def egcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x_prime, y_prime = egcd(b, a % b)
        return d, y_prime, x_prime - (a // b) * y_prime

def mul_inv(a, n):
    d, x, y = egcd(a, n)
    if d == 1:
        return x % n
    else:
        raise Exception("gcd(a, n)!= 1")
    
def mod_exp(a, b, n):
    res, b_2 = 1, bin(b)
    for d in b_2:
        res = (res * res) % n
        if d == "1":
            res = (res * a) % n
    return res

def witness(a, n):
    t, u = 0, n - 1
    while u % 2 == 0:
        u >>= 1
        t += 1
    x = mod_exp(a, u, n)
    for _ in range(t):
        x_prev = x
        x = mod_exp(x_prev, 2, n)
        if x == 1 and x_prev != 1 and x_prev != n - 1:
            return True
    if x != 1:
        return True
    return False

def Miller_Rabin_test(n, s = MILLER_RABIN_TRIAL_NUM):
    for _ in range(s):
        a = random.randrange(1, n - 1)
        if witness(a, n):
            return "composite"
    return "prime"

def generate_primes(bits):
    primes = []
    for _ in range(2):
        while True:
            num = random.randrange(2 ** (bits - 1), 2 ** bits)
            if Miller_Rabin_test(num) == "prime":
                primes.append(num)
                break
    return primes[0], primes[1]

def generate_keys(bits):
    p, q = generate_primes(int(bits/2))
    n = p * q
    phi_n = (p - 1) * (q - 1)
    while True:
        e = random.getrandbits(bits)
        if gcd(e, phi_n) == 1:
            break
    d = mul_inv(e, phi_n)
    return (e, n), (d, n)

def encrypt(P, text):
    e, n = P[0], P[1]
    M = int(binascii.hexlify(text.encode()), 16)
    return mod_exp(M, e, n)

def decrypt(S, cipher_text):
    d, n = S[0], S[1]
    M = mod_exp(cipher_text, d, n)
    return binascii.unhexlify(hex(M)[2:]).decode()

In [343]:
P1, S1 = generate_keys(2048)
P2, S2 = generate_keys(2048)

In [351]:
M = "I'm fine. How are you?"
cipher_text = encrypt(P1, M)
text = decrypt(S1, cipher_text)
assert(M == text)

In [ ]:
text = decrypt(S2, cipher_text)
assert(M == text)